In [1]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import bootstrap
import warnings
warnings.filterwarnings("ignore")

# 1(a)  Download the AReM data from: The dataset contains 7 folders that represent seven types of activities. Ineach folder, there are multiple files each of which represents an instant of a human performing an activity. Each file contains 6 time series collected from activities of the same person, which are called avgrss12, varrss12, avgrss13, varrss13,vgrss23, and arrss23. There are 88 instances in the dataset, each of which contains 6 time series and each time series has 480 consecutive values.

Data downloaded and explored for irregularities on Excel.

# 1(b) Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1, 2, and 3 in other folders as test data and other datasets as train data.

## Test Data Split

#### Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1, 2, and 3 in other folders as test data

In [2]:
folders = ["bending1","bending2"]
test_datasets = []
for folder in folders:
    for num in range(1,3):
        bending_test_df = pd.read_csv(f"../data/AReM/{folder}/dataset{num}.csv",skiprows=4)
        test_datasets.append(bending_test_df)
        
folder_2 = ["cycling","lying","sitting","standing","walking"]
for folder in folder_2:
    for num in range(1,4):
        all_others_test_df = pd.read_csv(f"../data/AReM/{folder}/dataset{num}.csv",skiprows=4)
        test_datasets.append(all_others_test_df)        
print("The Number of Datasets in the Test Data:", len(test_datasets))
print()
print(test_datasets)

The Number of Datasets in the Test Data: 19

[     # Columns: time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  \
0                  0      39.25       0.43      22.75       0.43      33.75   
1                250      39.25       0.43      23.00       0.00      33.00   
2                500      39.25       0.43      23.25       0.43      33.00   
3                750      39.50       0.50      23.00       0.71      33.00   
4               1000      39.50       0.50      24.00       0.00      33.00   
..               ...        ...        ...        ...        ...        ...   
475           118750      43.33       0.47      25.00       0.00      30.00   
476           119000      43.50       0.50      25.50       0.50      30.00   
477           119250      43.50       0.50      24.75       0.43      30.00   
478           119500      43.50       0.50      24.33       0.47      30.00   
479           119750      43.50       0.50      24.25       0.43      30.00   

     

The test data consists of 19 instances (19 datasets) as specified by the instructions.

## Train Data Split

#### All other datasets as train

In [3]:
train_datasets = []
for num in range(3,8):
        bending1_train_df = pd.read_csv(f"../data/AReM/bending1/dataset{num}.csv",skiprows=4)
        train_datasets.append(bending1_train_df)
for num in range(3,7):
        bending2_train_df = pd.read_csv(f"../data/AReM/bending2/dataset{num}.csv",skiprows=4)
        train_datasets.append(bending2_train_df)        
        
folders = ["cycling","lying","sitting","standing","walking"]
for folder in folders:
    for num in range(4,16):
        all_others_train_df = pd.read_csv(f"../data/AReM/{folder}/dataset{num}.csv",skiprows=4)
        train_datasets.append(all_others_train_df)        
print("The Number of Datasets in the Train Data:", len(train_datasets))
print()    
print(train_datasets)

The Number of Datasets in the Train Data: 69

[     # Columns: time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  \
0                  0      42.00       0.71      21.25       0.43      30.00   
1                250      41.50       0.50      20.25       1.48      31.25   
2                500      41.50       0.50      14.25       1.92      33.00   
3                750      40.75       0.83      15.75       0.43      33.00   
4               1000      40.00       0.71      20.00       2.74      32.75   
..               ...        ...        ...        ...        ...        ...   
475           118750      42.50       0.50      20.00       0.82      36.00   
476           119000      42.67       0.47      21.00       0.00      36.33   
477           119250      44.33       0.94      21.00       0.00      36.33   
478           119500      45.25       0.43      26.75       1.79      36.00   
479           119750      47.25       0.83      29.75       0.43      35.25   

    

The train data consists of 69 instances (69 datasets) as specified by the instructions.

# 1(c)(i) Research what types of time-domain features are usually used in time series classification and list them (examples are minimum, maximum, mean, etc).

Some examples of time-domain features are
- mean
- median
- mode
- maximum
- minimum
- standard deviation
- skewness
- interquartile range
- variance
- range
- kurtosis

# 1(c)(ii) Extract the time-domain features minimum, maximum, mean, median, stan-dard deviation, first quartile, and third quartile for all of the 6 time series in each instance. You are free to normalize/standardize features or use them directly.

In [4]:
def folderFeatureDataFrame(folder):
    columns = ["min1", "max1", "mean1", "median1", "std1", "firstquartile1", "thirdquartile1",
               "min2", "max2", "mean2", "median2", "std2", "firstquartile2", "thirdquartile2",
               "min3", "max3", "mean3", "median3", "std3", "firstquartile3", "thirdquartile3",
               "min4", "max4", "mean4", "median4", "std4", "firstquartile4", "thirdquartile4",
               "min5", "max5", "mean5", "median5", "std5", "firstquartile5", "thirdquartile5",
               "min6", "max6", "mean6", "median6", "std6", "firstquartile6", "thirdquartile6"]
    folder_df = pd.DataFrame(columns=columns)
    files = glob.glob(f"../data/AReM/{folder}/*.csv")
    for filepath in files:
        df = pd.read_csv(filepath, skiprows=4)
        df = df.drop(columns="# Columns: time")
        td_features_data = []
        for column in df.columns:
            minimum = df[column].min()
            maximum = df[column].max()
            mean = df[column].mean()
            median = df[column].median()
            stdev = df[column].std()
            firstquartile = df[column].quantile(0.25)
            thirdquartile = df[column].quantile(0.75)
            td_features_data.extend([minimum, maximum, mean, median, stdev, firstquartile, thirdquartile])

        folder_df = pd.concat([folder_df, pd.DataFrame([td_features_data], columns=columns)], ignore_index=True)
    
    return folder_df

In [5]:
bending_one = folderFeatureDataFrame("bending1")
bending_two = folderFeatureDataFrame("bending2")
cycling = folderFeatureDataFrame("cycling")
lying = folderFeatureDataFrame("lying")
sitting = folderFeatureDataFrame("sitting")
standing = folderFeatureDataFrame("standing")
walking = folderFeatureDataFrame("walking")
print(bending_one.shape)
print(bending_two.shape)
print(cycling.shape)
print(lying.shape)
print(sitting.shape)
print(standing.shape)
print(walking.shape)

(7, 42)
(6, 42)
(15, 42)
(15, 42)
(15, 42)
(15, 42)
(15, 42)


In [6]:
bending_one.head()

,min1,max1,mean1,median1,std1,firstquartile1,thirdquartile1,min2,max2,mean2,...,std5,firstquartile5,thirdquartile5,min6,max6,mean6,median6,std6,firstquartile6,thirdquartile6
0,36.25,48.00,43.969125,44.50,1.618364,43.31,44.67,0.0,1.50,0.413125,...,3.318301,20.5000,23.75,0.0,2.96,0.555312,0.49,0.487826,0.00,0.83
1,37.00,48.00,43.454958,43.25,1.386098,42.50,45.00,0.0,1.58,0.378083,...,2.488862,22.2500,24.00,0.0,5.26,0.679646,0.50,0.622534,0.43,0.87
2,33.00,47.75,42.179812,43.50,3.670666,39.15,45.00,0.0,3.00,0.696042,...,3.849448,30.4575,36.33,0.0,2.18,0.613521,0.50,0.524317,0.00,1.00
3,33.00,45.75,41.678063,41.75,2.243490,41.33,42.75,0.0,2.83,0.535979,...,2.411026,28.4575,31.25,0.0,1.79,0.383292,0.43,0.389164,0.00,0.50
4,37.25,45.00,40.624792,40.50,1.476967,39.25,42.00,0.0,1.30,0.358604,...,2.188449,33.0000,36.00,0.0,1.92,0.570583,0.43,0.582915,0.00,1.30


In [7]:
bending_two.head()

,min1,max1,mean1,median1,std1,firstquartile1,thirdquartile1,min2,max2,mean2,...,std5,firstquartile5,thirdquartile5,min6,max6,mean6,median6,std6,firstquartile6,thirdquartile6
0,25.00,47.50,31.058250,29.71,4.829794,27.5000,31.8125,0.0,6.38,0.405458,...,7.853427,9.0,18.3125,0.0,4.92,1.098104,0.94,0.831480,0.50,1.30
1,27.50,33.00,29.881938,30.00,1.153837,29.0000,30.2700,0.0,1.00,0.256437,...,1.745970,17.0,19.0000,0.0,6.40,0.701625,0.71,0.481103,0.47,0.94
2,19.00,45.50,30.938104,29.00,7.684146,26.7500,38.0000,0.0,6.40,0.467167,...,5.845911,15.0,20.8125,0.0,6.73,1.107354,0.83,1.080842,0.47,1.30
3,12.75,51.00,24.562958,24.25,3.737514,23.1875,26.5000,0.0,6.87,0.590833,...,3.693786,20.5,27.0000,0.0,4.97,0.700188,0.50,0.693720,0.43,0.87
4,0.00,42.75,27.464604,28.00,3.583582,25.5000,30.0000,0.0,7.76,0.449708,...,5.053642,15.0,20.7500,0.0,6.76,1.122125,0.83,1.012342,0.47,1.30


In [8]:
cycling.head()

,min1,max1,mean1,median1,std1,firstquartile1,thirdquartile1,min2,max2,mean2,...,std5,firstquartile5,thirdquartile5,min6,max6,mean6,median6,std6,firstquartile6,thirdquartile6
0,26.75,44.75,36.863375,36.33,3.555787,34.50,39.7500,0.0,11.68,2.757312,...,2.655906,15.00,18.6700,0.0,8.81,2.773313,2.59,1.569919,1.6400,3.6325
1,25.33,45.00,37.114313,36.25,3.710385,34.50,40.2500,0.0,10.84,2.730000,...,2.847876,14.75,18.5000,0.0,9.50,3.076354,2.77,1.824534,1.7000,4.0375
2,19.00,44.00,36.228396,36.00,3.528617,34.00,39.0000,0.0,12.28,2.831687,...,3.166655,14.00,18.0625,0.0,9.98,3.480688,3.34,1.827769,2.1025,4.5500
3,26.50,44.33,36.687292,36.00,3.529404,34.25,39.3725,0.0,12.89,2.973042,...,2.978238,14.67,18.5000,0.0,8.19,3.073313,2.69,1.629675,1.9125,4.0875
4,24.25,45.00,37.177042,36.25,3.581301,34.50,40.2500,0.0,8.58,2.374208,...,2.890347,17.95,21.7500,0.0,9.34,2.921729,2.50,1.852600,1.5000,3.9000


In [9]:
lying.head()

,min1,max1,mean1,median1,std1,firstquartile1,thirdquartile1,min2,max2,mean2,...,std5,firstquartile5,thirdquartile5,min6,max6,mean6,median6,std6,firstquartile6,thirdquartile6
0,39.0,40.00,39.433792,39.5,0.208558,39.33,39.50,0.0,1.00,0.422104,...,3.321622,9.0000,12.33,0.0,4.06,0.582708,0.50,0.438442,0.43,0.83
1,0.0,40.67,39.506188,39.5,1.817498,39.50,39.67,0.0,1.00,0.496479,...,2.336680,11.3300,13.00,0.0,3.19,0.622917,0.50,0.498889,0.43,0.87
2,34.0,51.00,42.706063,40.5,3.537476,40.25,48.00,0.0,4.85,0.519813,...,4.253807,1.0000,8.00,0.0,4.97,0.549312,0.47,0.648403,0.00,0.83
3,39.0,41.00,39.667833,39.5,0.280158,39.50,39.75,0.0,1.00,0.583604,...,4.097351,1.6275,9.33,0.0,3.49,0.635938,0.50,0.608399,0.00,0.83
4,23.5,30.00,27.716375,27.5,1.442253,27.00,29.00,0.0,1.79,0.363687,...,4.074511,5.5000,10.75,0.0,4.50,0.734271,0.71,0.613688,0.43,1.00


In [10]:
sitting.head()

,min1,max1,mean1,median1,std1,firstquartile1,thirdquartile1,min2,max2,mean2,...,std5,firstquartile5,thirdquartile5,min6,max6,mean6,median6,std6,firstquartile6,thirdquartile6
0,27.50,50.75,38.888375,36.75,3.642901,36.25,42.0,0.0,6.06,0.383438,...,4.978530,14.0000,20.75,0.0,5.17,0.840208,0.82,0.750636,0.43,1.090
1,27.00,38.67,32.905979,30.25,3.505776,30.00,37.0,0.0,2.49,0.456750,...,3.736307,16.5000,20.75,0.0,5.12,0.791354,0.71,0.755739,0.43,1.000
2,42.50,46.00,44.753958,45.00,0.520309,44.75,45.0,0.0,1.12,0.153917,...,4.127515,17.5000,21.69,0.0,4.30,0.764083,0.70,0.652091,0.43,1.000
3,25.50,45.75,36.560063,36.50,5.915081,30.00,42.5,0.0,12.19,0.422167,...,2.863600,15.5000,19.00,0.0,6.26,0.854833,0.83,0.717603,0.43,1.145
4,33.25,48.00,42.363563,43.00,2.068247,42.00,43.5,0.0,4.44,0.492563,...,4.956821,10.1875,17.69,0.0,6.02,1.034021,0.83,0.985627,0.47,1.250


In [11]:
standing.head()

,min1,max1,mean1,median1,std1,firstquartile1,thirdquartile1,min2,max2,mean2,...,std5,firstquartile5,thirdquartile5,min6,max6,mean6,median6,std6,firstquartile6,thirdquartile6
0,35.50,48.00,46.224938,46.00,1.748315,45.25,48.00,0.0,4.50,0.312354,...,2.931581,12.0000,15.2500,0.0,6.00,0.882583,0.83,0.668423,0.47,1.12
1,34.50,47.75,44.471146,45.00,1.772553,45.00,45.25,0.0,2.18,0.290479,...,2.612390,12.0000,14.8125,0.0,4.32,0.927375,0.83,0.756436,0.47,1.22
2,30.00,46.67,42.648521,42.75,2.395338,41.50,45.00,0.0,2.95,0.402833,...,3.141679,10.6275,14.2500,0.0,4.64,0.917354,0.83,0.709638,0.47,1.12
3,36.00,47.50,43.720021,45.00,2.384105,43.00,45.00,0.0,1.92,0.366708,...,3.289138,11.3100,15.5425,0.0,6.18,1.039687,0.83,0.916657,0.47,1.22
4,33.33,48.00,44.334729,45.00,2.476940,42.25,46.50,0.0,3.90,0.432958,...,5.401794,9.3300,17.7500,0.0,5.02,0.933000,0.83,0.673609,0.47,1.25


In [12]:
walking.head()

,min1,max1,mean1,median1,std1,firstquartile1,thirdquartile1,min2,max2,mean2,...,std5,firstquartile5,thirdquartile5,min6,max6,mean6,median6,std6,firstquartile6,thirdquartile6
0,18.33,47.67,34.333042,34.75,4.948770,31.25,38.0000,0.0,12.48,4.396958,...,3.000493,13.7500,18.00,0.0,8.01,3.261583,2.980,1.617290,2.050,4.320
1,21.50,51.00,34.935812,35.50,4.645944,32.00,38.0625,0.0,12.21,4.115750,...,3.192058,14.2375,18.25,0.0,10.21,3.280021,3.015,1.700918,2.120,4.500
2,18.00,46.00,35.193333,36.00,4.751868,32.00,38.7500,0.0,16.20,4.321021,...,3.207642,14.2500,18.50,0.0,8.50,3.241958,3.015,1.769182,1.885,4.440
3,20.75,46.25,34.763333,35.29,4.742208,31.67,38.2500,0.0,12.68,4.223792,...,3.174681,14.2500,18.33,0.0,9.39,3.288271,3.270,1.647528,2.050,4.305
4,19.33,43.50,34.227771,35.50,4.889576,30.50,37.7500,0.0,14.50,3.995729,...,3.092094,14.7500,18.67,0.0,9.74,3.394125,3.100,1.792090,2.105,4.425


# 1(c)(ii) New Dataset with Time Series Features for All 88 Instances

In [13]:
all_folders_df = pd.concat([bending_one,bending_two,cycling,lying,sitting,standing,walking], ignore_index=True)
all_folders_df.index = all_folders_df.index + 1
all_folders_df

,min1,max1,mean1,median1,std1,firstquartile1,thirdquartile1,min2,max2,mean2,...,std5,firstquartile5,thirdquartile5,min6,max6,mean6,median6,std6,firstquartile6,thirdquartile6
1,36.25,48.00,43.969125,44.500,1.618364,43.31,44.67,0.0,1.50,0.413125,...,3.318301,20.5000,23.75,0.00,2.96,0.555312,0.490,0.487826,0.0000,0.8300
2,37.00,48.00,43.454958,43.250,1.386098,42.50,45.00,0.0,1.58,0.378083,...,2.488862,22.2500,24.00,0.00,5.26,0.679646,0.500,0.622534,0.4300,0.8700
3,33.00,47.75,42.179812,43.500,3.670666,39.15,45.00,0.0,3.00,0.696042,...,3.849448,30.4575,36.33,0.00,2.18,0.613521,0.500,0.524317,0.0000,1.0000
4,33.00,45.75,41.678063,41.750,2.243490,41.33,42.75,0.0,2.83,0.535979,...,2.411026,28.4575,31.25,0.00,1.79,0.383292,0.430,0.389164,0.0000,0.5000
5,37.25,45.00,40.624792,40.500,1.476967,39.25,42.00,0.0,1.30,0.358604,...,2.188449,33.0000,36.00,0.00,1.92,0.570583,0.430,0.582915,0.0000,1.3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,19.75,45.50,34.322750,35.250,4.752477,31.00,38.00,0.0,13.47,4.456333,...,3.119856,13.5000,17.75,0.00,9.67,3.432563,3.200,1.732727,2.1575,4.5650
85,19.25,44.00,34.473188,35.000,4.796705,31.25,38.00,0.0,13.86,4.359312,...,3.156320,13.7300,17.75,0.43,9.00,3.340458,3.090,1.699114,2.1200,4.3750
86,23.50,46.25,34.873229,35.250,4.531720,31.75,38.25,0.0,14.82,4.380583,...,3.131076,13.7500,18.00,0.00,9.51,3.424646,3.270,1.690960,2.1700,4.5000
87,18.33,45.75,34.599875,35.125,4.731790,31.50,38.00,0.0,15.37,4.398833,...,2.905688,14.0000,18.25,0.00,8.86,3.289542,3.015,1.680170,2.1200,4.2600


# 1(c)(iii) Estimate the standard deviation of each of the time-domain features you extracted from the data. 

In [14]:
columns = ["min1", "max1", "mean1", "median1", "std1", "firstquartile1", "thirdquartile1", 
           "min2", "max2", "mean2", "median2", "std2", "firstquartile2", "thirdquartile2",
            "min3", "max3", "mean3", "median3", "std3", "firstquartile3", "thirdquartile3",
            "min4", "max4", "mean4", "median4", "std4", "firstquartile4", "thirdquartile4",
            "min5", "max5", "mean5", "median5", "std5", "firstquartile5", "thirdquartile5",
            "min6", "max6", "mean6", "median6", "std6", "firstquartile6", "thirdquartile6"]
stdev_list = []
for column in all_folders_df.columns:
    stdev = all_folders_df[column].std()
    stdev_list.append(stdev)
stdev_df = pd.DataFrame([stdev_list],columns=columns)
stdev_df.transpose()

,0
min1,9.569975
max1,4.394362
mean1,5.335718
median1,5.440054
std1,1.772153
firstquartile1,6.153590
thirdquartile1,5.138925
min2,0.000000
max2,5.062729
mean2,1.574164


The time series feature minimum tends to, but not in all cases, have a standard deviation that is very close to 0. The time series feature standard deviation has a standard deviation usually in the range of 0.5 to 2. First quartile and third quartile time series features have varying standard deviations between time series with some standard deviations being far greater than others for these features.

# 1(c)(iii)Then, use Python’s bootstrapped or any other method to build a 90% bootsrap confidence interval for the standard deviation of each feature.

In [15]:
interval_number = 0
for td_feature in all_folders_df.columns:
    interval_number += 1
    feature_data = ((all_folders_df[td_feature]),)
    bootstrap_data = bootstrap(feature_data, np.std, confidence_level=0.9, random_state=24)
    confidence_interval = bootstrap_data.confidence_interval
    print(f'{interval_number}) Confidence interval for {td_feature}:')
    print(confidence_interval)
    print()

1) Confidence interval for min1:
ConfidenceInterval(low=8.516313622836554, high=11.180022986006906)

2) Confidence interval for max1:
ConfidenceInterval(low=3.4910267710357306, high=5.458139923744808)

3) Confidence interval for mean1:
ConfidenceInterval(low=4.794157444508065, high=5.979219726317231)

4) Confidence interval for median1:
ConfidenceInterval(low=4.87041128933334, high=6.086918499304847)

5) Confidence interval for std1:
ConfidenceInterval(low=1.5969007612278718, high=1.9855604233683117)

6) Confidence interval for firstquartile1:
ConfidenceInterval(low=5.649319992561867, high=6.726396166949477)

7) Confidence interval for thirdquartile1:
ConfidenceInterval(low=4.440128747688505, high=5.9478453811723355)

8) Confidence interval for min2:
ConfidenceInterval(low=nan, high=nan)

9) Confidence interval for max2:
ConfidenceInterval(low=4.672439494816932, high=5.439769488989891)

10) Confidence interval for mean2:
ConfidenceInterval(low=1.409146673684522, high=1.7116371113907114

# 1(c)(iv) Use your judgement to select the three most important time-domain features (one option may be min, mean, and max).

From my judgment, the three best time-domain features are median, standard deviation, and the maximum. The median will be more rigid to any outliers and shows the center of the data. The standard deviation will help us understand the spread of the data. And the maximum in a time series can help us identify the peak which can be important for understanding time series data.

# ISLR 3.7.4) I collect a set of data (n = 100 observations) containing a single predictor and a quantitative response. I then fit a linear regression model to the data, as well as a separate cubic regression, i.e. Y = β0 +β1X +β2X2 +β3X3 +ε.

#### a) Suppose that the true relationship between X and Y is linear, i.e. Y = β0 + β1X + ε. Consider the training residual sum of squares (RSS) for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

I believe that the Training RSS for the cubic regression will be lower than the Training RSS for linear regression. Using cubic regression may result in the model being too overfit and therefore too closely following the trends of the data. As a result, the RSS could appear lower for cubic regression even though X and Y have a linear relationship.

#### (b) Answer (a) using test rather than training RSS.

Because the test data is unseen and still has a linear relationship, the overfit cubic regression model may not fit the test data as well and result in a higher RSS. Since the two variables have a linear relationship, the linear regression model will be better able to fit unseen data resulting in a lower RSS as it was previously fit to train data which had a similar linear relationship.

#### c) Suppose that the true relationship between X and Y is not linear, but we don’t know how far it is from linear. Consider the training RSS for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

I would expect the cubic regression RSS to be lower because the data doesn't have a linear relationship. Although we don't know how far it is from linear, the fact that is not linear indicates that cubic regression as opposed to linear regression will produce a smaller RSS value as it is able to better fit non linear data.

#### d) Answer (c) using test rather than training RSS.

In this case, the test data RSS is highly dependent on how far the relationship of the variable deviates from linearity.

If the relationship isn't far off from being linear then I would expect linear regression model to have a lower test RSS as the cubic regression model would run the risk of overfitting/ being specific to the data.

However if the relationship is far from linear then I would expect the cubic regression test RSS to be lower as it is better able to fit the specifities of the non linear data.

# References

For bootstrapping:
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bootstrap.html

glob: https://docs.python.org/3/library/glob.html